# Secondo progetto di Social Computing

In [24]:
import pandas as pd

# importaimo il file .csv usando la libreria Pandas
df = pd.read_csv("group_5-Palma-Sacchet-Sagliocca.csv")

In [75]:
documents = []

randomId = df.sample().iloc()[0]['id']

for index, row in df.iterrows():
    document = {
        'id': row['id'],
        'statement': row['statement'],
    }
    if (row['id'] == randomId):
        document['explanation'] = row['explanation_human']
    else:
        document['explanation'] = row['explanation_model']
    document['label'] = row['label']
    
    documents.append(document)

print(f'RandomID = {randomId}')
print(documents)

RandomID = 1520
[{'id': 25842, 'statement': 'Saw II sold over 3 million units.', 'explanation': 'The claim is that Saw II sold over 3 million units. The evidence says that Saw II was released to DVD and topped charts its first week, selling more than 3 million units. So we can infer that the claim is true.', 'label': 'SUPPORTS'}, {'id': 41947, 'statement': 'Matthew Gray Gubler is not an actor.', 'explanation': 'The claim is that Matthew Gray Gubler is not an actor. The evidence says that he is an American actor. So, the claim is false.', 'label': 'REFUTES'}, {'id': 1520, 'statement': 'Azithromycin is sold under different trade names.', 'explanation': '"The relevant information about the claim is lacking in the context."', 'label': 'SUPPORTS'}]
